## Observations and Insights 

In [17]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Combine the data into a single dataset
mouse_data = pd.merge(mouse_metadata, study_results, on="Mouse ID", how="outer")

# Display the data table for preview
mouse_data

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1
...,...,...,...,...,...,...,...,...
1888,z969,Naftisol,Male,9,30,25,63.145652,2
1889,z969,Naftisol,Male,9,30,30,65.841013,3
1890,z969,Naftisol,Male,9,30,35,69.176246,4
1891,z969,Naftisol,Male,9,30,40,70.314904,4


In [18]:
# Checking the number of mice.
mouse_counts_before = mouse_data["Mouse ID"].nunique()
mouse_counts_before

249

In [19]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 
mouse_dupes = mouse_data[mouse_data.duplicated(['Mouse ID', 'Timepoint'])]
mouse_dupes_ID = mouse_dupes.iloc[0, 0]
mouse_dupes_ID

'g989'

In [20]:
# Optional: Get all the data for the duplicate mouse ID. 
mouse_dupes_all = mouse_data[mouse_data['Mouse ID'] == mouse_dupes_ID]
mouse_dupes_all

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
908,g989,Propriva,Female,21,26,0,45.000000,0
909,g989,Propriva,Female,21,26,0,45.000000,0
910,g989,Propriva,Female,21,26,5,48.786801,0
911,g989,Propriva,Female,21,26,5,47.570392,0
912,g989,Propriva,Female,21,26,10,51.745156,0
913,g989,Propriva,Female,21,26,10,49.880528,0
914,g989,Propriva,Female,21,26,15,51.325852,1
915,g989,Propriva,Female,21,26,15,53.442020,0
916,g989,Propriva,Female,21,26,20,55.326122,1
917,g989,Propriva,Female,21,26,20,54.657650,1


In [21]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
mouse_data.drop(mouse_data[mouse_data['Mouse ID'] == mouse_dupes_ID].index, inplace = True)
mouse_data

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1
...,...,...,...,...,...,...,...,...
1888,z969,Naftisol,Male,9,30,25,63.145652,2
1889,z969,Naftisol,Male,9,30,30,65.841013,3
1890,z969,Naftisol,Male,9,30,35,69.176246,4
1891,z969,Naftisol,Male,9,30,40,70.314904,4


In [22]:
# Checking the number of mice in the clean DataFrame.
mouse_counts_after = mouse_data["Mouse ID"].nunique()
mouse_counts_after

248

## Summary Statistics

In [35]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen
# This method is the most straighforward, creating multiple series and putting them all together at the end.
regimen = mouse_data["Drug Regimen"]


regimen_tumorvol
regimen = regimen.drop_duplicates()
regimen_tumorvol_mean = []
regimen_tumorvol_median = []
regimen_tumorvol_var = []
regimen_tumorvol_std = []
regimen_tumorvol_sem = []

for drug in regimen:
    print(drug)
    regimen_tumorvol = mouse_data.loc[mouse_data['Drug Regimen']==drug, 'Tumor Volume (mm3)']
    regimen_tumorvol_mean.append(np.mean(regimen_tumorvol))
    regimen_tumorvol_median.append(np.median(regimen_tumorvol))
    regimen_tumorvol_var.append(np.var(regimen_tumorvol))
    regimen_tumorvol_std.append(np.std(regimen_tumorvol))
    sample = regimen_tumorvol.sample(10)
    regimen_tumorvol_sem.append(st.sem(sample))
    
print(f"Mean: {regimen_tumorvol_mean}")
print(f"Median: {regimen_tumorvol_median}")
print(f"Variance: {regimen_tumorvol_var}")
print(f"Standard deviation: {regimen_tumorvol_std}")
print(f"SEM: {regimen_tumorvol_sem}")

d = {'Mean':regimen_tumorvol_mean,'Med':days}
>d
{'Day': [31, 30, 31, 30], 'Month': ['Jan', 'Apr', 'Mar', 'June']}


Ramicane
Capomulin
Infubinol
Placebo
Ceftamin
Stelasyn
Zoniferol
Ketapril
Propriva
Naftisol
Mean: [40.2167450667105, 40.67574114100001, 52.88479510859551, 54.03358078635358, 52.59117180960677, 54.23314911988949, 53.236506551593415, 55.23563764047869, 52.32092986, 54.331564658333306]
Median: [40.67323554, 41.557808879999996, 51.82058438, 52.28893409, 51.77615728000001, 52.43173664, 51.818479325, 53.698742644999996, 50.446266345, 52.509284609999995]
Variance: [23.38369209265624, 24.839295580601572, 42.88638815058418, 60.830137727102, 39.06944594028265, 59.122105531524184, 48.266688601532294, 68.18893042567763, 43.55571562779733, 65.8177075949599]
Standard deviation: [4.83566873272521, 4.9839036488079875, 6.54876997233711, 7.799367777397217, 6.250555650522812, 7.6890900327362655, 6.947423162693654, 8.257658894970028, 6.599675418366977, 8.112811324994555]
SEM: [1.5360378758544444, 2.2277310146288043, 2.1672669606089157, 2.6690307538688254, 1.8340767805634983, 2.57279019583016, 2.2990706510

In [8]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# This method produces everything in a single groupby function


## Bar and Pie Charts

In [9]:
# Generate a bar plot showing the total number of mice for each treatment throughout the course of the study using pandas. 



In [10]:
# Generate a bar plot showing the total number of mice for each treatment throughout the course of the study using pyplot.



In [11]:
# Generate a pie plot showing the distribution of female versus male mice using pandas



In [12]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot



## Quartiles, Outliers and Boxplots

In [13]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse


# Merge this group df with the original dataframe to get the tumor volume at the last timepoint


In [14]:
# Put treatments into a list for for loop (and later for plot labels)


# Create empty list to fill with tumor vol data (for plotting)


# Calculate the IQR and quantitatively determine if there are any potential outliers. 

    
    # Locate the rows which contain mice on each drug and get the tumor volumes
    
    
    # add subset 
    
    
    # Determine outliers using upper and lower bounds
    

In [15]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest


## Line and Scatter Plots

In [16]:
# Generate a line plot of time point versus tumor volume for a mouse treated with Capomulin


In [17]:
# Generate a scatter plot of mouse weight versus average tumor volume for the Capomulin regimen


## Correlation and Regression

In [18]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen
